In [ ]:
# Import the required packages and libraries
import json
import requests
import datetime
import pandas as pd
import os
from dotenv import load_dotenv 
from pathlib import Path
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from pyprediktormapclient.auth_client import AUTH_CLIENT
from pyprediktormapclient.analytics_helper import AnalyticsHelper
from pyprediktormapclient.shared import *

In [ ]:
# Consider obtaining the envrionment variables from .env file if you are running this locally from source.
dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

In [ ]:
username = os.environ["USERNAME"]
password = os.environ["PASSWORD"]
opcua_rest_url = os.environ["OPC_UA_REST_URL"]
opcua_server_url = os.environ["OPC_UA_SERVER_URL"]
model_index_url = os.environ["MODEL_INDEX_URL"]
ory_url = os.environ["ORY_URL"]

In [ ]:
# Getting ory bearer token
auth_client = AUTH_CLIENT(rest_url=ory_url, username=username, password=password)
auth_client.request_new_ory_token()

In [ ]:
url = model_index_url
headers = {"Content-Type": "application/json",
            "Accept": "application/json"
            }
auth_client = auth_client
session: requests.Session = None

In [ ]:
if auth_client is not None:
    if auth_client.token is not None:
        headers["Authorization"] = f"Bearer {auth_client.token.session_token}"
    if hasattr(auth_client, 'session_token'):
        headers["Cookie"] = f"ory_kratos_session={auth_client.session_token}"

body = {"Connection": {"Url": url, "AuthenticationType": 1}}

In [ ]:
def get_object_types() -> str:
        content = request_from_api(url, "GET", "query/object-types", headers=headers, session=session)

        return content

In [ ]:
object_types = get_object_types()
object_types

In [ ]:
def get_namespace_array() -> str:
        """Get the namespace array

        Returns:
            str: the JSON returned from the server
        """
        content = request_from_api(url, "GET", "query/namespaces", headers=headers, session=session)

        return content

In [ ]:
name_space_array = get_namespace_array()
name_space_array

In [ ]:
def get_object_type_id_from_name(type_name: str) -> str:
        """Get object type id from type name

        Args:
            type_name (str): type name

        Returns:
            str: the type id that corresponds with the id (None if not found)
        """
        try:
            obj_type = next(
                item for item in object_types if item["BrowseName"]["Name"] == type_name
            )
        except StopIteration:
            obj_type = {}

        if obj_type:
            node_id = obj_type.get("NodeId", {})
            object_type_id = f'{node_id.get("Namespace")}:{node_id.get("IdType")}:{node_id.get("Id")}'
        else:
            object_type_id = None

        return object_type_id

In [ ]:
object_type_id = get_object_type_id_from_name("StringSetType")
object_type_id

In [ ]:
def get_objects_of_type(type_name: str) -> str:
        """Function to get all the types of an object

        Args:
            type_name (str): type name

        Returns:
            A json-formatted string with the objects (or None if the type is not found)
        """
        object_type_id = get_object_type_id_from_name(type_name)
        if object_type_id is None:
            return None
        
        namespace, id_type, id_ = object_type_id.split(':')

        body = json.dumps({"Id": int(id_), "IdType": int(id_type), "Namespace": int(namespace)})
        content = request_from_api(url, "POST", "query/objects-of-type-with-variables-and-properties", body, headers=headers, session=session)

        return content

In [ ]:
objects_of_type = get_objects_of_type("SiteType")
objects_of_type

In [ ]:
def get_object_descendants(
        type_name: str,
        ids: list,
        domain: str,
    ) -> str:
        """A function to get object descendants

        Args:
            type_name (str): type_name of a descendant
            ids (list): a list of ids you want the descendants for
            domain (str): PV_Assets or PV_Serves

        Returns:
            A json-formatted string with descendats data of selected object (or None if the type is not found)
        """
        if type_name is None or not ids:
            raise ValidationError("type_name and ids cannot be None or empty")
    
        id = get_object_type_id_from_name(type_name)
        body = json.dumps(
            {
                "typeId": id,
                "objectIds": ids,
                "domain": domain,
            }
        )
        content = request_from_api(url, "POST", "query/object-descendants?include_members=true", body, headers=headers, session=session)

        return content

In [ ]:
object_descendants = get_object_descendants("StringSetType", ['3:1:Enterprise.JO-GL'], "PV_Assets")
object_descendants